In [1]:
import brightway2 as bw
import pandas as pd
import numpy as np

In [2]:
bw.projects.set_current('psh_lca')

In [3]:
bw.databases

Databases dictionary with 3 object(s):
	biosphere3
	ecoinvent3.7.1 cut-off
	ecoinvent3.8 cons

Database dictionary must contain both ecoinvent3.8 cons and ecoinvent3.7.1 cut-off for the following notebook to work without revision. If your databases have different names, edit the "eco" variable defined in the first cell of each section to match your local ecoinvent database names.

Ecoinvent contains grid mixes for the U.S. electric regions listed in the cell below.

Source: Ecoinvent electricity writeup at
https://proyectaryproducir.com.ar/public_html/Seminarios_Posgrado/Material_de_referencia/ecoinvent-electricity%20datasets%20201305%20final.pdf

### Database name definition

In [4]:
eco7 = "ecoinvent3.7.1 cut-off"
eco8 = "ecoinvent3.8 cons"

### List of U.S. grid regions

In [5]:
elec_locs = ['FRCC', 'ASCC', 'HICC', 
             'MRO', 'NPCC', 'RFC', 
             'SERC', 'SPP', 'TRE',
             'WECC']

### Processes that probably use U.S. electricity

For reference (used at the end of this section): A list of all processes in Ecoinvent with a US location, *other than* processes directly involved in electricity generation and transmission. These processes will use a "market group for electricity" input that combines "market for electricity" processes and ultimately uses U.S. grid mixes.

In [6]:
usdb = [i for i in bw.Database(eco7) if (
    i.as_dict()['location'] == 'US'
)]
usdb[:5]

['cement, all types to generic market for cement, unspecified' (kilogram, US, None),
 'market for polyethylene, high density, granulate, recycled' (kilogram, US, None),
 'market for water, decarbonised' (kilogram, US, None),
 'uranium production, diffusion, enriched 4.2%' (unit, US, None),
 'natural gas production' (cubic meter, US, None)]

In [7]:
usdb = [i for i in bw.Database(eco8) if (
    i.as_dict()['location'] == 'US'
)]
usdb[:5]

['uranium production, diffusion, enriched 3.9%' (unit, US, None),
 'polyethylene terephthalate, granulate, amorphous, recycled to generic market for amorphous PET granulate' (kilogram, US, None),
 '1,1-difluoroethane production, HFC-152a' (kilogram, US, None),
 'market for polyethylene terephthalate, granulate, amorphous, recycled' (kilogram, US, None),
 'water production, decarbonised' (kilogram, US, None)]

### All electricity-related activites

In [8]:
elec = [i for i in bw.Database(eco7) 
             for loc in elec_locs 
             if (loc in i.as_dict()['location'])
        and ('electricity' in i.as_dict()['name'])]
elec

['market for electricity, high voltage' (kilowatt hour, US-WECC, None),
 'electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, panel, mounted' (kilowatt hour, US-SERC, None),
 'market for electricity, medium voltage' (kilowatt hour, US-ASCC, None),
 'electricity voltage transformation from medium to low voltage' (kilowatt hour, US-NPCC, None),
 'electricity production, hydro, pumped storage' (kilowatt hour, US-NPCC, None),
 'electricity production, nuclear, boiling water reactor' (kilowatt hour, US-TRE, None),
 'electricity production, deep geothermal' (kilowatt hour, US-WECC, None),
 'electricity production, natural gas, conventional power plant' (kilowatt hour, US-WECC, None),
 'electricity production, photovoltaic, 570kWp open ground installation, multi-Si' (kilowatt hour, US-RFC, None),
 'market for electricity, high voltage' (kilowatt hour, US-RFC, None),
 'electricity production, lignite' (kilowatt hour, US-FRCC, None),
 'electricity production, hydro,

In [9]:
elec = [i for i in bw.Database(eco8) 
             for loc in elec_locs 
             if (loc in i.as_dict()['location'])
        and ('electricity' in i.as_dict()['name'])]
elec

['electricity production, hard coal' (kilowatt hour, US-MRO, None),
 'electricity production, wind, >3MW turbine, onshore' (kilowatt hour, US-MRO, None),
 'electricity production, nuclear, pressure water reactor' (kilowatt hour, US-MRO, None),
 'electricity production, deep geothermal' (kilowatt hour, US-WECC, None),
 'electricity production, hard coal' (kilowatt hour, US-SERC, None),
 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, US-RFC, None),
 'electricity production, hydro, reservoir, alpine region' (kilowatt hour, US-NPCC, None),
 'electricity production, nuclear, pressure water reactor' (kilowatt hour, US-TRE, None),
 'electricity production, hard coal' (kilowatt hour, US-ASCC, None),
 'market for electricity, high voltage' (kilowatt hour, US-TRE, None),
 'electricity production, lignite' (kilowatt hour, US-MRO, None),
 'electricity voltage transformation from high to medium voltage' (kilowatt hour, US-TRE, None),
 'market for electricity, medium voltage'

We ignore "production mix" processes because they "are only of informative
character and it is NOT recommended to use the production mixes in your life
cycle analysis, because the electricity markets represent electricity supply" (per source above).

### All activities that produce electricity from a single fuel type

In [10]:
elecfuel7 = [i for i in bw.Database(eco7) 
             for loc in elec_locs 
             if (loc in i.as_dict()['location'])
             and ('electricity production' in i.as_dict()['name'])]
elecfuel7

['electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted' (kilowatt hour, US-SERC, None),
 'electricity production, oil' (kilowatt hour, US-RFC, None),
 'electricity production, deep geothermal' (kilowatt hour, US-WECC, None),
 'electricity production, wind, >3MW turbine, onshore' (kilowatt hour, US-ASCC, None),
 'electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, panel, mounted' (kilowatt hour, US-NPCC, None),
 'electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, panel, mounted' (kilowatt hour, US-RFC, None),
 'electricity production, lignite' (kilowatt hour, US-FRCC, None),
 'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted' (kilowatt hour, US-TRE, None),
 'electricity production, lignite' (kilowatt hour, US-ASCC, None),
 'electricity production, hydro, run-of-river' (kilowatt hour, US-HICC, None),
 'electricity production, photovoltaic, 3kW

In [11]:
elecfuel8 = [i for i in bw.Database(eco8) 
             for loc in elec_locs 
             if (loc in i.as_dict()['location'])
             and ('electricity production' in i.as_dict()['name'])]
elecfuel8

['electricity production, wind, >3MW turbine, onshore' (kilowatt hour, US-MRO, None),
 'electricity production, nuclear, boiling water reactor' (kilowatt hour, US-NPCC, None),
 'electricity production, photovoltaic, 570kWp open ground installation, multi-Si' (kilowatt hour, US-SERC, None),
 'electricity production, natural gas, combined cycle power plant' (kilowatt hour, US-WECC, None),
 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, US-HICC, None),
 'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted' (kilowatt hour, US-RFC, None),
 'electricity production, natural gas, combined cycle power plant' (kilowatt hour, US-ASCC, None),
 'electricity production, hard coal' (kilowatt hour, US-HICC, None),
 'electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, panel, mounted' (kilowatt hour, US-SERC, None),
 'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mount

## Electricity market groups
There are "market for electricity" and "market group" processes. The "market groups" combine grid mixes from the various U.S. regions into a national grid mix for use as an input to U.S. activities.

In [12]:
elecmarketgroup7 = [i for i in bw.Database(eco7) 
             if ('US' in i.as_dict()['location'])
             and ('market group for electricity' in i.as_dict()['name'])]
elecmarketgroup7

['market group for electricity, low voltage' (kilowatt hour, US, None),
 'market group for electricity, medium voltage' (kilowatt hour, US, None),
 'market group for electricity, high voltage' (kilowatt hour, US, None)]

In [13]:
elecmarketgroup8 = [i for i in bw.Database(eco8) 
             if ('US' in i.as_dict()['location'])
             and ('market group for electricity' in i.as_dict()['name'])]
elecmarketgroup8

['market group for electricity, high voltage' (kilowatt hour, US, None),
 'market group for electricity, medium voltage' (kilowatt hour, US, None),
 'market group for electricity, low voltage' (kilowatt hour, US, None)]

In [14]:
market1_7=elecmarketgroup7[0]
[i for i in bw.Database(eco7).get(market1_7.key[1]).exchanges()]

[Exchange: 1.0 kilowatt hour 'market group for electricity, low voltage' (kilowatt hour, US, None) to 'market group for electricity, low voltage' (kilowatt hour, US, None)>,
 Exchange: 0.00126295779431007 kilowatt hour 'market for electricity, low voltage' (kilowatt hour, US-ASCC, None) to 'market group for electricity, low voltage' (kilowatt hour, US, None)>,
 Exchange: 0.0549542401770875 kilowatt hour 'market for electricity, low voltage' (kilowatt hour, US-FRCC, None) to 'market group for electricity, low voltage' (kilowatt hour, US, None)>,
 Exchange: 0.00222184618979088 kilowatt hour 'market for electricity, low voltage' (kilowatt hour, US-HICC, None) to 'market group for electricity, low voltage' (kilowatt hour, US, None)>,
 Exchange: 0.111492323654095 kilowatt hour 'market for electricity, low voltage' (kilowatt hour, US-MRO, None) to 'market group for electricity, low voltage' (kilowatt hour, US, None)>,
 Exchange: 0.0636235121033833 kilowatt hour 'market for electricity, low v

In [15]:
market1_8=elecmarketgroup8[0]
[i for i in bw.Database(eco8).get(market1_8.key[1]).exchanges()]

[Exchange: 1.0 kilowatt hour 'market group for electricity, high voltage' (kilowatt hour, US, None) to 'market group for electricity, high voltage' (kilowatt hour, US, None)>,
 Exchange: 0.00103199174406605 kilowatt hour 'market for electricity, high voltage' (kilowatt hour, US-ASCC, None) to 'market group for electricity, high voltage' (kilowatt hour, US, None)>,
 Exchange: 0.00206398348813209 kilowatt hour 'market for electricity, high voltage' (kilowatt hour, US-HICC, None) to 'market group for electricity, high voltage' (kilowatt hour, US, None)>,
 Exchange: 0.0670794633642931 kilowatt hour 'market for electricity, high voltage' (kilowatt hour, US-MRO, None) to 'market group for electricity, high voltage' (kilowatt hour, US, None)>,
 Exchange: 0.0629514963880289 kilowatt hour 'market for electricity, high voltage' (kilowatt hour, US-NPCC, None) to 'market group for electricity, high voltage' (kilowatt hour, US, None)>,
 Exchange: 0.251805985552116 kilowatt hour 'market for electric

## Market for electricity (region-specific grid mixes by voltage)

The "market for electricity" processes within each voltage level will tell us the grid mix.

In [16]:
elecmixes7 = [i for i in bw.Database(eco7) 
             for loc in elec_locs 
             if (loc in i.as_dict()['location'])
             and ('market for electricity' in i.as_dict()['name'])]
elecmixes7

['market for electricity, medium voltage' (kilowatt hour, US-MRO, None),
 'market for electricity, low voltage' (kilowatt hour, US-NPCC, None),
 'market for electricity, high voltage' (kilowatt hour, US-TRE, None),
 'market for electricity, high voltage' (kilowatt hour, US-WECC, None),
 'market for electricity, low voltage' (kilowatt hour, US-MRO, None),
 'market for electricity, low voltage' (kilowatt hour, US-TRE, None),
 'market for electricity, medium voltage' (kilowatt hour, US-RFC, None),
 'market for electricity, low voltage' (kilowatt hour, US-ASCC, None),
 'market for electricity, low voltage' (kilowatt hour, US-SERC, None),
 'market for electricity, medium voltage' (kilowatt hour, US-WECC, None),
 'market for electricity, medium voltage' (kilowatt hour, US-ASCC, None),
 'market for electricity, medium voltage' (kilowatt hour, US-HICC, None),
 'market for electricity, high voltage' (kilowatt hour, US-MRO, None),
 'market for electricity, high voltage' (kilowatt hour, US-RFC, N

In [17]:
elecmixes8 = [i for i in bw.Database(eco8) 
             for loc in elec_locs 
             if (loc in i.as_dict()['location'])
             and ('market for electricity' in i.as_dict()['name'])]
elecmixes8

['market for electricity, low voltage' (kilowatt hour, US-SERC, None),
 'market for electricity, medium voltage' (kilowatt hour, US-HICC, None),
 'market for electricity, medium voltage' (kilowatt hour, US-TRE, None),
 'market for electricity, medium voltage' (kilowatt hour, US-NPCC, None),
 'market for electricity, high voltage' (kilowatt hour, US-NPCC, None),
 'market for electricity, low voltage' (kilowatt hour, US-MRO, None),
 'market for electricity, medium voltage' (kilowatt hour, US-RFC, None),
 'market for electricity, medium voltage' (kilowatt hour, US-ASCC, None),
 'market for electricity, high voltage' (kilowatt hour, US-ASCC, None),
 'market for electricity, high voltage' (kilowatt hour, US-MRO, None),
 'market for electricity, high voltage' (kilowatt hour, US-RFC, None),
 'market for electricity, low voltage' (kilowatt hour, US-WECC, None),
 'market for electricity, high voltage' (kilowatt hour, US-TRE, None),
 'market for electricity, high voltage' (kilowatt hour, US-WECC

What do the exchanges look like for these "market for electricity" processes? 

### Low voltage process.

In [18]:
mix7_1 = [i for i in elecmixes7 if 'low' in i.as_dict()['name']][0]
mix7_1

'market for electricity, low voltage' (kilowatt hour, US-NPCC, None)

In [19]:
[i for i in bw.Database(eco7).get(mix7_1.key[1]).exchanges()]

[Exchange: 1.0 kilowatt hour 'market for electricity, low voltage' (kilowatt hour, US-NPCC, None) to 'market for electricity, low voltage' (kilowatt hour, US-NPCC, None)>,
 Exchange: 8.74048809653223e-08 kilometer 'market for distribution network, electricity, low voltage' (kilometer, GLO, None) to 'market for electricity, low voltage' (kilowatt hour, US-NPCC, None)>,
 Exchange: 7.79e-09 kilogram 'market for sulfur hexafluoride, liquid' (kilogram, RoW, None) to 'market for electricity, low voltage' (kilowatt hour, US-NPCC, None)>,
 Exchange: 0.0026099707227365 kilowatt hour 'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted' (kilowatt hour, US-NPCC, None) to 'market for electricity, low voltage' (kilowatt hour, US-NPCC, None)>,
 Exchange: 0.00209563184950604 kilowatt hour 'electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, panel, mounted' (kilowatt hour, US-NPCC, None) to 'market for electricity, low voltage' (k

In [20]:
mix8_1 = [i for i in elecmixes8 if 'low' in i.as_dict()['name']][0]
mix8_1

'market for electricity, low voltage' (kilowatt hour, US-SERC, None)

In [21]:
[i for i in bw.Database(eco8).get(mix8_1.key[1]).exchanges()]

[Exchange: 8.74048809653223e-08 kilometer 'market for distribution network, electricity, low voltage' (kilometer, GLO, None) to 'market for electricity, low voltage' (kilowatt hour, US-SERC, None)>,
 Exchange: 7.79e-09 kilogram 'market for sulfur hexafluoride, liquid' (kilogram, RoW, None) to 'market for electricity, low voltage' (kilowatt hour, US-SERC, None)>,
 Exchange: 1.0 kilowatt hour 'market for electricity, low voltage' (kilowatt hour, US-SERC, None) to 'market for electricity, low voltage' (kilowatt hour, US-SERC, None)>,
 Exchange: 0.330521516968893 kilowatt hour 'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted' (kilowatt hour, US-SERC, None) to 'market for electricity, low voltage' (kilowatt hour, US-SERC, None)>,
 Exchange: 0.669478483031106 kilowatt hour 'electricity voltage transformation from medium to low voltage' (kilowatt hour, US-SERC, None) to 'market for electricity, low voltage' (kilowatt hour, US-SERC, None)>,
 Excha

In this low-voltage process, there's a tiny fraction of solar electricity being added to electricity that's converted from medium voltage. The solar electricity is coming from two types of roof installations and a multi-SI ground installation. Total electricity from these additional generating processes is less than 0.006 kWh per kWh of low voltage electricity.

### Medium voltage process.

In [22]:
mix7_2 = [i for i in elecmixes7 if 'medium' in i.as_dict()['name']][0]
mix7_2

'market for electricity, medium voltage' (kilowatt hour, US-MRO, None)

In [23]:
[i for i in bw.Database(eco7).get(mix7_2.key[1]).exchanges()]

[Exchange: 1.86277676887616e-08 kilometer 'market for transmission network, electricity, medium voltage' (kilometer, GLO, None) to 'market for electricity, medium voltage' (kilowatt hour, US-MRO, None)>,
 Exchange: 1.29e-07 kilogram 'market for sulfur hexafluoride, liquid' (kilogram, RoW, None) to 'market for electricity, medium voltage' (kilowatt hour, US-MRO, None)>,
 Exchange: 1.0 kilowatt hour 'market for electricity, medium voltage' (kilowatt hour, US-MRO, None) to 'market for electricity, medium voltage' (kilowatt hour, US-MRO, None)>,
 Exchange: 0.998580457982689 kilowatt hour 'electricity voltage transformation from high to medium voltage' (kilowatt hour, US-MRO, None) to 'market for electricity, medium voltage' (kilowatt hour, US-MRO, None)>,
 Exchange: 0.00141954201731061 kilowatt hour 'electricity, from municipal waste incineration to generic market for electricity, medium voltage' (kilowatt hour, RoW, None) to 'market for electricity, medium voltage' (kilowatt hour, US-MRO,

In [24]:
mix8_2 = [i for i in elecmixes8 if 'medium' in i.as_dict()['name']][0]
mix8_2

'market for electricity, medium voltage' (kilowatt hour, US-HICC, None)

In [25]:
[i for i in bw.Database(eco8).get(mix8_2.key[1]).exchanges()]

[Exchange: 1.29e-07 kilogram 'market for sulfur hexafluoride, liquid' (kilogram, RoW, None) to 'market for electricity, medium voltage' (kilowatt hour, US-HICC, None)>,
 Exchange: 1.0 kilowatt hour 'market for electricity, medium voltage' (kilowatt hour, US-HICC, None) to 'market for electricity, medium voltage' (kilowatt hour, US-HICC, None)>,
 Exchange: 1.86277676887616e-08 kilometer 'market for transmission network, electricity, medium voltage' (kilometer, GLO, None) to 'market for electricity, medium voltage' (kilowatt hour, US-HICC, None)>,
 Exchange: 1.0 kilowatt hour 'electricity voltage transformation from high to medium voltage' (kilowatt hour, US-HICC, None) to 'market for electricity, medium voltage' (kilowatt hour, US-HICC, None)>,
 Exchange: 0.0033 kilowatt hour 'market for electricity, medium voltage' (kilowatt hour, US-HICC, None) to 'market for electricity, medium voltage' (kilowatt hour, US-HICC, None)>,
 Exchange: 1.29e-07 kilogram 'Sulfur hexafluoride' (kilogram, Non

At medium voltage, there's again some electricity generation being added to electricity converted from high voltage. The new generation is only from municipal waste incineration, 0.001 kWh per kWh medium voltage electricity. There is also some parasitic consumption, 0.003 kWh per kWh.

### High voltage process - where most of the grid mix definition takes place.

In [26]:
mix7_3 = [i for i in elecmixes7 if 'high' in i.as_dict()['name']][0]
mix7_3

'market for electricity, high voltage' (kilowatt hour, US-TRE, None)

In [27]:
[i for i in bw.Database(eco7).get(mix7_3.key[1]).exchanges()]

[Exchange: 6.58209848825026e-09 kilometer 'market for transmission network, electricity, high voltage' (kilometer, GLO, None) to 'market for electricity, high voltage' (kilowatt hour, US-TRE, None)>,
 Exchange: 1.0 kilowatt hour 'market for electricity, high voltage' (kilowatt hour, US-TRE, None) to 'market for electricity, high voltage' (kilowatt hour, US-TRE, None)>,
 Exchange: 3.17e-10 kilometer 'market for transmission network, long-distance' (kilometer, GLO, None) to 'market for electricity, high voltage' (kilowatt hour, US-TRE, None)>,
 Exchange: 0.000454781829741937 kilowatt hour 'electricity production, hydro, reservoir, non-alpine region' (kilowatt hour, US-TRE, None) to 'market for electricity, high voltage' (kilowatt hour, US-TRE, None)>,
 Exchange: 0.00181913606218204 kilowatt hour 'electricity production, hydro, run-of-river' (kilowatt hour, US-TRE, None) to 'market for electricity, high voltage' (kilowatt hour, US-TRE, None)>,
 Exchange: 0.230933914519861 kilowatt hour 'e

In [28]:
mix8_3 =  [i for i in elecmixes8 if 'high' in i.as_dict()['name']][0]
mix8_3

'market for electricity, high voltage' (kilowatt hour, US-NPCC, None)

In [29]:
[i for i in bw.Database(eco8).get(mix8_3.key[1]).exchanges()]

[Exchange: 6.58209848825026e-09 kilometer 'market for transmission network, electricity, high voltage' (kilometer, GLO, None) to 'market for electricity, high voltage' (kilowatt hour, US-NPCC, None)>,
 Exchange: 1.0 kilowatt hour 'market for electricity, high voltage' (kilowatt hour, US-NPCC, None) to 'market for electricity, high voltage' (kilowatt hour, US-NPCC, None)>,
 Exchange: 3.17e-10 kilometer 'market for transmission network, long-distance' (kilometer, GLO, None) to 'market for electricity, high voltage' (kilowatt hour, US-NPCC, None)>,
 Exchange: 0.0427272259671179 kilowatt hour 'electricity production, deep geothermal' (kilowatt hour, RoW, None) to 'market for electricity, high voltage' (kilowatt hour, US-NPCC, None)>,
 Exchange: 0.0236680116769202 kilowatt hour 'electricity production, hydro, reservoir, alpine region' (kilowatt hour, US-NPCC, None) to 'market for electricity, high voltage' (kilowatt hour, US-NPCC, None)>,
 Exchange: 0.0946720467076809 kilowatt hour 'electri

### Exchanges for (inputs to) high-voltage electricity processes

The high voltage process is where the vast majority of the grid mix definition takes place. In this process, the following power generator types are used (fuels or renewables)

For ecoinvent3.7.1 cut-off:

*BN: The cells below will break at some index because the exchanges past that point are in biosphere, not ecoinvent. *

In [36]:
exkey = [i.as_dict()['input'][1] for i in bw.Database(eco7).get(mix7_3.key[1]).exchanges()]
for i in list(np.arange(len(exkey))):
    print(i, bw.Database(eco7).get(exkey[i]))

0 'market for transmission network, electricity, high voltage' (kilometer, GLO, None)
1 'market for electricity, high voltage' (kilowatt hour, US-TRE, None)
2 'market for transmission network, long-distance' (kilometer, GLO, None)
3 'electricity production, hydro, reservoir, non-alpine region' (kilowatt hour, US-TRE, None)
4 'electricity production, hydro, run-of-river' (kilowatt hour, US-TRE, None)
5 'electricity production, lignite' (kilowatt hour, US-TRE, None)
6 'electricity production, natural gas, combined cycle power plant' (kilowatt hour, US-TRE, None)
7 'electricity production, natural gas, conventional power plant' (kilowatt hour, US-TRE, None)
8 'electricity production, nuclear, boiling water reactor' (kilowatt hour, US-TRE, None)
9 'electricity production, nuclear, pressure water reactor' (kilowatt hour, US-TRE, None)
10 'electricity production, oil' (kilowatt hour, US-TRE, None)
11 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, US-TRE, None)
12 'ele

ActivityDatasetDoesNotExist: <Model: ActivityDataset> instance matching query does not exist:
SQL: SELECT "t1"."id", "t1"."data", "t1"."code", "t1"."database", "t1"."location", "t1"."name", "t1"."product", "t1"."type" FROM "activitydataset" AS "t1" WHERE (("t1"."database" = ?) AND ("t1"."code" = ?)) ORDER BY Random() LIMIT ? OFFSET ?
Params: ['ecoinvent3.7.1 cut-off', '20185046-64bb-4c09-a8e7-e8a9e144ca98', 1, 0]

For ecoinvent 3.8 cons:

In [37]:
exkey = [i.as_dict()['input'][1] for i in bw.Database(eco8).get(mix8_3.key[1]).exchanges()]
for i in list(np.arange(len(exkey))):
    print(i, bw.Database(eco8).get(exkey[i]))

0 'market for transmission network, electricity, high voltage' (kilometer, GLO, None)
1 'market for electricity, high voltage' (kilowatt hour, US-NPCC, None)
2 'market for transmission network, long-distance' (kilometer, GLO, None)
3 'electricity production, deep geothermal' (kilowatt hour, RoW, None)
4 'electricity production, hydro, reservoir, alpine region' (kilowatt hour, US-NPCC, None)
5 'electricity production, hydro, run-of-river' (kilowatt hour, US-NPCC, None)
6 'electricity production, natural gas, combined cycle power plant' (kilowatt hour, US-NPCC, None)
7 'electricity production, wind, >3MW turbine, onshore' (kilowatt hour, US-NPCC, None)
8 'market for electricity, high voltage' (kilowatt hour, US-NPCC, None)


ActivityDatasetDoesNotExist: <Model: ActivityDataset> instance matching query does not exist:
SQL: SELECT "t1"."id", "t1"."data", "t1"."code", "t1"."database", "t1"."location", "t1"."name", "t1"."product", "t1"."type" FROM "activitydataset" AS "t1" WHERE (("t1"."database" = ?) AND ("t1"."code" = ?)) ORDER BY Random() LIMIT ? OFFSET ?
Params: ['ecoinvent3.8 cons', '2058d91e-bb9f-4fce-8e53-131825b14a23', 1, 0]